In [ ]:
import pandas as pd
from pandas import DataFrame

In [ ]:
# カラム名を修正・タイムスタンプは必要ないので削除
nms = ["TimeStamp", "Grade", "Faculty", "Sex", "Living", "2F3N", "2N3F", "2F3F", 
       "FreqG", "AvePayG", "WhenG", "PeopleG", "ReasonUseG", "ReasonNotUseG", "FreqMenu", 
       "LikeG", "DislikeG", "ImprsG", "FreqP", "GvsP", "WhereP", "EvsW", "ReasonEvsW", 
       "App", "ReasonUseApp", "RepositApp", "ReasonNotUseApp", "AppFeature"]
df_forms = pd.read_csv("forms.csv", header=0, names=nms, usecols=nms[1:])

In [ ]:
for col in df_forms.columns:
    print(f"{col}: {df_forms[col].unique()}")

In [ ]:
df_forms.replace({"Grade": {'学部1年': "B1", '学部2年': "B2", '学部3年': "B3", '学部4年': "B4", '院生': "M"}, 
                  "Faculty": {'商学部': "Com", '経済学部': "Econ", '社会学部': "Soc", '法学部': "Law", 
                              'ソーシャル・データ・サイエンス学部': "SDS", '大学院': "Grad"}, 
                  "Sex": {'男性': "M", '女性': "F"}, 
                  "Living": {'一人暮らし（自炊をしない）': "Alone(NoCook)", 
                              '一人暮らし（自炊をする）': "Alone(Cook)", 
                              '寮暮らし（食事が出ない）': "Dormitory(NoMeal)", 
                              '寮暮らし（食事が出る）': "Dormitory(Meal)", 
                              '実家暮らし': "WithParents"}, 
                  "2F3N": {'なし': 0, '週１回': 1, '週２回': 2, '週３回': 3, '週４回': 4, '週５回': 5}, 
                  "2N3F": {'なし': 0, '週１回': 1, '週２回': 2, '週３回': 3, '週４回': 4, '週５回': 5}, 
                  "2F3F": {'なし': 0, '週１回': 1, '週２回': 2, '週３回': 3, '週４回': 4, '週５回': 5}, 
                  "FreqG": {'0回': "0", '１回': "1", '２〜３回': "2-3", '４回以上': "4+"}, 
                  "AvePayG": {'利用しない': "0", '〜300円': "1-299", '300円〜500円': "300-499", 
                              '500円〜800円': "500-799", '800円〜1000円': "800-999", '1000円〜': "1000+"}, 
                  "PeopleG": {'利用しない': "0", '1人': "1", '2～3人': "2-3", '4人以上': "4+"}, 
                  "FreqP": {'週０回': "0", '月1回程度。部活後にアイス食べたりはしちゃいます。': "1/M", 
                            '月2〜3回': "2-3/M", '週１回': "1/W", '週２〜３回': "2-3/W", '週４回以上': "4+/W"}, 
                  "GvsP": {'学食': "Gakushoku", 'パンショップ': "PanShop", '好き嫌い無くどちらも利用する': "Both", 'どちらも好きではない': "NoneOfThem"}, 
                  "WhereP": {'学食内の席': "InGakushoku", '次の授業の教室': "NxtClass", 'その他教室': "OtherClass", 
                             '研究室': "Lab", '部室': "Clubroom", '外のベンチ': "Outside", '食べ歩き': "Outside", 
                             '利用しない': "DontUse", '買わない': "DontUse"}, 
                  "EvsW": {'東学食': "East", '西学食': "West", 'どちらも好きではない': "NoneOfThem", 'どちらも好き。': "Both"}, 
                  "App": {'はい': "Use", 'いいえ': "DontUse"},
                  "RepositApp": {'アプリ導入時に学食カード？の残金を移行したのが残っているため使っている。使い切ったら使用しないと思う。': "DontReposit", 
                                 '入学した時にチャージしただけ': "DontReposit", 'クレカ入金（自分のお金）': "CreditCard(own)", 
                                 'クレカ入金（親のお金）': "CreditCard(prt)", 'コンビニ入金（自分のお金）': "ConvStore(own)", 
                                 'コンビニ入金（親のお金）': "ConvStore(prt)", '生協レジでチャージ（自分のお金）': "CoopCasher(own)", 
                                 '生協レジでチャージ（親のお金）': "CoopCasher(prt)"}}, 
                inplace=True)
df_forms.fillna("Unanswered", inplace=True)
df_WhenG = df_forms[["WhenG"]].copy()
df_ReasonUseG = df_forms[["ReasonUseG"]].copy()
df_ReasonNotUseG = df_forms[["ReasonNotUseG"]].copy()
df_FreqMenu = df_forms[["FreqMenu"]].copy()
df_LikeG = df_forms[["LikeG"]].copy()
df_DislikeG = df_forms[["DislikeG"]].copy()
df_ImprsG = df_forms[["ImprsG"]].copy()
df_ReasonEvsW = df_forms[["ReasonEvsW"]].copy()
df_ReasonUseApp = df_forms[["ReasonUseApp"]].copy()
df_ReasonNotUseApp = df_forms[["ReasonNotUseApp"]].copy()
df_AppFeature = df_forms[["AppFeature"]].copy()
df_forms.drop(columns=["WhenG", "ReasonUseG", "ReasonNotUseG", "FreqMenu", "LikeG", "DislikeG", "ImprsG", 
                       "ReasonEvsW", "ReasonUseApp", "ReasonNotUseApp", "AppFeature"], 
                       inplace=True)

In [ ]:
def split_and_check_values(df: DataFrame) -> set:
    """
    ユニークな回答を返す
    """
    r = len(df)
    splited_ans = set()
    for i in range(r):
        splited_ans = splited_ans | set(str(df.iloc[i, 0]).split(", "))
    return set(splited_ans)

In [ ]:
def split_and_to_dummy(df: DataFrame, 
                      original_to_new: dict[str, list]) -> DataFrame:
    """
    複数回答を分割し、新たなカラムを指定してダミー変数化\\
    なお、回答のうち文字列""は無視されるため、original_to_newに含める必要はない
    """
    r = len(df)

    new_column = []
    for value_list in original_to_new.values():
        new_column += value_list
    new_column = list(set(new_column))
    
    n = len(new_column)
    dummy = [[0] * n for _ in range(r)]
    
    for i in range(r):
        splited_ans = str(df.iloc[i, 0]).split(", ")
        for ans in splited_ans:
            if ans == "":
                continue
            new_ans_list = original_to_new[ans]
            for new_ans in new_ans_list:
                dummy[i][new_column.index(new_ans)] = 1
    return pd.DataFrame(data=dummy, columns=new_column)

In [ ]:
split_and_check_values(df_WhenG)

In [ ]:
df_dummy_WhenG = split_and_to_dummy(df_WhenG, 
                                    {'11:30~11:45': ['11:30-11:45'], '11:45~12:00': ['11:45-12:00'], 
                                     '12:00~12:15': ['12:00-12:15'], '12:15~12:30': ['12:15-12:30'], 
                                     '12:30~12:45': ['12:30-12:45'], '12:45~13:00': ['12:45-13:00'], 
                                     '13:00~13:15': ['13:00-13:15'], '13:15~13:30': ['13:15-13:30'], 
                                     '13:30~13:45': ['13:30-13:45'], '13:45~14:00': ['13:45-14:00'], 
                                     '14:00~14:15': ['14:00-14:15'], '14:15~14:30': ['14:15-14:30'], 
                                     'パンショップのみ利用': ["PanShop"],'利用しない': ["DontUse"],'夜営業': ["Night"]})
df_dummy_WhenG = df_dummy_WhenG.reindex(columns=['11:30-11:45', '11:45-12:00', '12:00-12:15', 
                                                 '12:15-12:30', '12:30-12:45', '12:45-13:00', 
                                                 '13:00-13:15', '13:15-13:30', '13:30-13:45', 
                                                 '13:45-14:00', '14:00-14:15', '14:15-14:30', 
                                                 "Night", "PanShop", "DontUse"])

In [ ]:
split_and_check_values(df_ReasonUseG)

In [ ]:
df_dummy_ReasonUseG = split_and_to_dummy(df_ReasonUseG, 
                                          {'人間関係の構築のため': ['HumanRelation'], 
                                           '他の人が行きたいと言うから': ["HumanRelation"], 
                                           '価格が安いから': ["Cheap"], 
                                           '友人に誘われるから': ["Friend"], 
                                           '友達と行くのに1番誘いやすいから': ["Friend"], 
                                           '友達と食べたいから': ["Friend"], 
                                           '大学の外に行かなくていいので': ["Close"], 
                                           '近いから': ["Close"], 
                                           '大学内でご飯を食べる場所が生協しかないから。': ["OnlyCoopInUniv"], 
                                           '学食なら親が費用を負担してくれるから': ["ParentsToPay"], 
                                           '手っ取り早く量を食べれる': ["CanEatQuickly"], 
                                           '授業の教室に近いから\u3000友達と合流しやすいから': ["Close", "Friend"], 
                                           '支払いが便利だから': ["EasyToPay"], 
                                           '時間の節約になるから': ["TimeSave"], 
                                           '栄養バランスが良いから': ["Nutrition"], 
                                           '食べたことのないメニューが出ることがあるから。': ["Menu"], 
                                           '利用していない': ["DontUse"]})
df_dummy_ReasonUseG = df_dummy_ReasonUseG.reindex(columns=['HumanRelation', 'Cheap', "Friend", "Close", 
                                                             "OnlyCoopInUniv", "ParentsToPay", "CanEatQuickly", 
                                                             "EasyToPay", "TimeSave", "Nutrition", "Menu", "DontUse"])                                         

In [ ]:
split_and_check_values(df_ReasonNotUseG)

In [ ]:
df_dummy_ReasonNotUseG = split_and_to_dummy(df_ReasonNotUseG,
                                            {'下宿まで5分程度で家に帰って作り置きなどを食べる方がコストパフォーマンスが良いように感じる。学食(生協？)は値上げをしないようがんばっていることは伝わるものの、私が体育会ということもあり1メニューでは空腹を満たしきれないのであまり積極的に使おうとは思わない。': ["PreferHome", "SmallMeal"],
                                             '人がたくさんいて、落ち着かない': ["Crowded", "CantBeRelaxed"],
                                             '利用している': ["Use"], '利用しない理由はない': ["Use"],
                                             '利用する必要がない。': ["NoNeedToUse"],
                                             '外食やコンビニを好むから': ["PreferOut"],
                                             '家に帰りたいから': ["PreferHome"],
                                             '家のご飯の方が安くて美味しい': ["PreferHome", "Expensive", "NotDelicious"],
                                             '家帰って食べた方が節約になる': ["PreferHome", "Expensive"],
                                             '座席が確保しにくいから': ["FewSeats"],
                                             '混雑していて、並ぶのが嫌': ["Crowded", "LongQueue"],
                                             '清潔感がないから': ["Dirty"],
                                             '自炊や外食の方が、コストや質の点で妥当感がある。': ["PreferOut", "Expensive", "NotDelicious"],
                                             '食堂の値段がたかいから': ["Expensive"],
                                             '高いしさほど美味しくない': ["Expensive", "NotDelicious"],
                                             '高価であるため': ["Expensive"]})
df_dummy_ReasonNotUseG = df_dummy_ReasonNotUseG.reindex(columns=['PreferHome', 'SmallMeal', 'Crowded', 
                                                                   'CantBeRelaxed', "NoNeedToUse", "PreferOut", 
                                                                   "Expensive", "NotDelicious", "FewSeats", 
                                                                   "LongQueue", "Dirty", "Use"])

In [ ]:
split_and_check_values(df_FreqMenu)

In [ ]:
df_dummy_FreqMenu = split_and_to_dummy(df_FreqMenu, 
                                        {'おかず': ["Okazu"], 
                                         'パン': ["Bread"], 
                                         '丼もの（カレー含む）': ["Don-mono"], 
                                         '利用しない': ["DontUse"], 
                                         '小鉢': ["Kobachi"], 
                                         '麺類': ["Noodle"]})
df_dummy_FreqMenu = df_dummy_FreqMenu.reindex(columns=['Okazu', 'Bread', 'Don-mono', 'Kobachi', 'Noodle', 'DontUse'])

In [ ]:
split_and_check_values(df_LikeG)

In [ ]:
df_dummy_LikeG = split_and_to_dummy(df_LikeG, 
                                    {'Unanswered': ["Unanswered"], 'パンが美味しい': ["DeliciousBread"], 
                                     'メニューのバリエーションが豊富': ["WideVarietyMenu"], 
                                     '利用しない': ["DontUse"], '安い': ["Cheap"], 
                                     '支払いが便利': ["ConvenientToPay"], '近い': ["Close"]})
df_dummy_LikeG = df_dummy_LikeG.reindex(columns=['DeliciousBread', 'WideVarietyMenu', 'Cheap', 
                                                   'ConvenientToPay', 'Close', 'DontUse', 'Unanswered'])

In [ ]:
split_and_check_values(df_DislikeG)

In [ ]:
df_dummy_DislikeG = split_and_to_dummy(df_DislikeG, 
                                       {'インテリアがいまいち': ["PoorInterior"], 'メニュー内容': ["PoorMenu"], 
                                        '並ぶのが嫌、時間がかかる': ["LongQueue"], 
                                        '人がたくさんいて、落ち着かない': ["Crowded", "CantBeRelaxed"], 
                                        '価格が高い': ["Expensive"], '座席が少ない': ["FewSeats"], 
                                        '価格が高いというより、無理して値下げするなら普通の価格でお腹いっぱい食べたいです。': ["SmallMeal"], 
                                        '利用しない': ["DontUse"], '動線が悪い': ["BadFlow"]})
df_dummy_DislikeG = df_dummy_DislikeG.reindex(columns=['PoorInterior', 'PoorMenu', 'LongQueue', "Crowded", "CantBeRelaxed", 
                                                         "Expensive", "FewSeats", "SmallMeal", "BadFlow", "DontUse"])

In [ ]:
split_and_check_values(df_ReasonEvsW)

In [ ]:
df_dummy_ReasonEvsW = split_and_to_dummy(df_ReasonEvsW, 
                                         {'Unanswered': ["Unanswered"], 
                                          'パンが充実しているから。': ["WideVarietyBread"], 
                                          'パンショップでしか学食(生協)を利用しないため。': ["OnlyUsePan-shop"], 
                                          'メニュー数が多い': ["WideVarietyMenu"], 
                                          '売店が充実している': ["RichSchoolStore"], 
                                          '席数が多い': ["MoreSeats"], '東は片付けが楽': ["EasyToClear"], 
                                          '混んでいない気がする': ["NotCrowded"], 
                                          '片付けが楽': ["EasyToClear"], '生協があるから': ["Coop"], 
                                          '立地': ["Location"], '西生協はパンの種類が多いから。': ["WideVarietyBread"], 
                                          '雰囲気': ["Atmosphere"]})
df_dummy_ReasonEvsW = df_dummy_ReasonEvsW.reindex(columns=['WideVarietyBread', 'OnlyUsePan-shop', 'WideVarietyMenu', 
                                                           'RichSchoolStore', 'MoreSeats', 'EasyToClear', 
                                                           'NotCrowded', 'Coop', 'Location', 'Atmosphere', 'Unanswered'])

In [ ]:
split_and_check_values(df_ReasonUseApp)

In [ ]:
df_dummy_ReasonUseApp = split_and_to_dummy(df_ReasonUseApp, 
                                           {'Unanswered': ["Unanswered"], 'ポイントが貯まる': ["Point"], 
                                            '学食ペイの残金を他で使えないから。': ["CantUseGakuPayElsewhere"], 
                                            '専用レジを利用し、迅速に会計を済ませたい': ["ToPayQuickly"], 
                                            '支払いが便利': ["ConvenientToPay"], 
                                            '教科書販売のときに利用することの方が多いです。': ["ToBuyTextbook"]})
df_dummy_ReasonUseApp = df_dummy_ReasonUseApp.reindex(columns=['Point', 'CantUseGakuPayElsewhere', 'ToPayQuickly', 
                                                               'ConvenientToPay', 'ToBuyTextbook', 'Unanswered'])

In [ ]:
split_and_check_values(df_ReasonNotUseApp)

In [ ]:
df_dummy_ReasonNotUseApp = split_and_to_dummy(df_ReasonNotUseApp, 
                                              {'使用が面倒だから': ["TroublesomeToUse"], 
                                               '使用方法を知らないから': ["DontKnowHowToUse"], 
                                               '入金が面倒だから': ["TroublesomeToReposit"], 
                                               '利用している': ["Use"], 
                                               '利用価値を感じないから': ["NotWorthUsing"], 
                                               '存在を知らないから': ["DontKnowApp"], 
                                               '学食を利用しないから': ["DontUseGakushoku"]})
df_dummy_ReasonNotUseApp = df_dummy_ReasonNotUseApp.reindex(columns=['TroublesomeToUse', 'DontKnowHowToUse', 
                                                                     'TroublesomeToReposit', 'NotWorthUsing', 
                                                                     'DontKnowApp', 'DontUseGakushoku', 'Use'])

In [ ]:
split_and_check_values(df_AppFeature)

In [ ]:
df_dummy_AppFeature = split_and_to_dummy(df_AppFeature, 
                                         {'Unanswered': ['Unanswered'], 
                                          'アプリの軽さ。': ["Responsiveness"], 
                                          'クレカとの紐付け': ["CreditCard"], 
                                          'クーポンの配布': ["Coupon"], 
                                          'ドーナツ情報。ミスドは1週間前にアプリでも通知してほしいです': ["DonutsInfo"], 
                                          '事前注文': ["PreOrder"], 
                                          '席から頼めたら便利。入金もっと簡単にして欲しいです🥺。(クレカ情報の記憶など)': ["PreOrder", "CreditCard"], 
                                          '混雑状況の確認': ["CrowdednessInfo"]})
df_dummy_AppFeature = df_dummy_AppFeature.reindex(columns=['Responsiveness', 'CreditCard', 'Coupon', 'DonutsInfo', 
                                                           'PreOrder', 'CrowdednessInfo', 'Unanswered'])

In [ ]:
dfs = [df_forms, df_dummy_WhenG, df_dummy_ReasonUseG, df_dummy_ReasonNotUseG, df_dummy_FreqMenu, 
       df_dummy_LikeG, df_dummy_DislikeG, df_dummy_ReasonEvsW, df_dummy_ReasonUseApp, 
       df_dummy_ReasonNotUseApp, df_dummy_AppFeature, df_ImprsG]
names = ["NonDummy", "WhenG", "ReasonUseG", "ReasonNotUseG", "FreqMenu", "LikeG", "DislikeG", 
         "ReasonEvsW", "ReasonUseApp", "ReasonNotUseApp", "AppFeature", "ImprsG"]
with pd.ExcelWriter("forms_cleaned.xlsx") as writer:
    for df, name in zip(dfs, names):
        df.to_excel(writer, sheet_name=name, index=False)